<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import function as fn

# Загрузка тренировочных данных данных

In [3]:
train = pd.read_parquet('data/input/data_fusion_train.parquet')
train = train[train.category_id != -1]

In [4]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6483226 entries, 1 to 45729281
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   receipt_id         int64  
 1   receipt_dayofweek  int32  
 2   receipt_time       object 
 3   item_name          object 
 4   item_quantity      float64
 5   item_price         int64  
 6   item_nds_rate      int32  
 7   category_id        int64  
 8   brands             object 
dtypes: float64(1), int32(2), int64(3), object(3)
memory usage: 445.2+ MB


# Сводные данные

In [6]:
# Общее число записей
count = train.shape[0]
count

6483226

In [7]:
# Популярные категории
print(f"Всего {train['category_id'].value_counts().count()} категорий")
np.round(train['category_id'].value_counts()/count, 6) * 100

Всего 98 категорий


203    20.1637
84     13.9206
80     10.3472
78      7.7616
71      6.5104
        ...   
106     0.0028
97      0.0017
46      0.0010
143     0.0004
121     0.0000
Name: category_id, Length: 98, dtype: float64

In [8]:
train.loc[train['category_id'] == 78, 'item_name'].value_counts()

Сметана 10                                          12042
Молоко Мудрая хозяйка 2,5% пленка 0,8 л БЗМЖ         7844
Сметана 10г, 16 ккал                                 6532
ЯЙЦО СТОЛОВОЕ 1КАТ. ПТИЦЕФАБРИКА ПИОНЕРСКОЕ 10ШТ     6358
Яйцо куриное КОЛЬЦОВСКОЕ СО 10шт                     5194
                                                    ...  
Йогурт двухсл 2,9-5,5% 180г голубика                    1
Майонез КАЛЬВЕ Классический м/у 200г                    1
Йогурт пит.Нежный  0.1 % лесные ягоды 285 г             1
Живой Ёгурт малиновый 200мл                             1
 БЗМЖ Сыр творожный сливочный                           1
Name: item_name, Length: 2870, dtype: int64

In [9]:
# Популярные Бренды
print(f"Всего {train['brands'].value_counts().count()} Брендов")
brands_serias = np.round(train['brands'].value_counts()/count, 6) * 100
brands_serias

Всего 2624 Брендов


кока-кола       0.8274
орбит           0.4102
пепси           0.2143
пепси-кола      0.1690
вискас          0.1414
                 ...  
loft            0.0000
master fruit    0.0000
sibo            0.0000
roberto         0.0000
neon            0.0000
Name: brands, Length: 2624, dtype: float64

# Выделение количества и единицы измерения

In [23]:
item_name_col_df = train[['item_name']].drop_duplicates()
item_name_col_df.item_name = item_name_col_df.item_name.str.lower()
item_name_col_df

,item_name
1,"молоко 3,2%,шт"
3,"компот из изюма, 114 ккал"
4,"макаронные изделия отварные (масло сливочное),..."
17,кофе капучино большой эден 18
40,хлеб на сыворотке 350г
...,...
45669181,"напиток энерг. ред булл 0,355л"
45681543,хеменгуэй дайкири
45690702,"пиво светлое ""халзан"" 4,5 % об, пл/б. 1,5 л(шт)"
45692298,экспресс педикюр


In [24]:
item_name_col_df_trnsform = fn.add_ed_izm(item_name_col_df)
item_name_col_df_trnsform['item_name_old'] = item_name_col_df['item_name']
item_name_col_df_trnsform

,item_name,ed_izm,col,item_name_old
1,"молоко 3,2%,шт",NaN,NaN,"молоко 3,2%,шт"
3,"компот из изюма, 114 ккал",NaN,NaN,"компот из изюма, 114 ккал"
4,"макаронные изделия отварные (масло сливочное),...",NaN,NaN,"макаронные изделия отварные (масло сливочное),..."
17,кофе капучино большой эден 18,NaN,NaN,кофе капучино большой эден 18
40,хлеб на сыворотке,gram,0.350,хлеб на сыворотке 350г
...,...,...,...,...
45669181,напиток энерг. ред булл,litr,355.000,"напиток энерг. ред булл 0,355л"
45681543,хеменгуэй дайкири,NaN,NaN,хеменгуэй дайкири
45690702,"пиво светлое ""халзан"" 4,5 % об, пл/б. (шт)",litr,15.000,"пиво светлое ""халзан"" 4,5 % об, пл/б. 1,5 л(шт)"
45692298,экспресс педикюр,NaN,NaN,экспресс педикюр


In [34]:
item_name_col_df_trnsform[item_name_col_df_trnsform['item_name'
                                                   ].str.contains('\d{1,4}.{0,2}мл')][:60]

,item_name,ed_izm,col,item_name_old
289,морс 200 мл,NaN,NaN,морс 200 мл
878,"сок яблочный, 250 мл,шт",NaN,NaN,"сок яблочный, 250 мл,шт"
1258,"контейнер д/анализов, 100 мл елмд",NaN,NaN,"контейнер д/анализов, 100 мл елмд"
1353,"сок яблочный 200 мл,шт",NaN,NaN,"сок яблочный 200 мл,шт"
1909,биогая капли детские 5мл фл с дозатором,NaN,NaN,биогая капли детские 5мл фл с дозатором
3531,"кокт молоч речка шоколад 2% бзмж 0,2 мл",NaN,NaN,"кокт молоч речка шоколад 2% бзмж 0,2 мл"
5443,морс клюква 200 мл,NaN,NaN,морс клюква 200 мл
7909,средство скор 2мл шт,NaN,NaN,средство скор 2мл шт
8301,масло подсолнечное урожайное 870 мл раф. дез. ...,NaN,NaN,масло подсолнечное урожайное 870 мл раф. дез. ...
9096,напиток ягодный 200 мл.,NaN,NaN,напиток ягодный 200 мл.
